<a href="https://colab.research.google.com/github/MagicPolygon/spotify-extended-streaming-history-analysis/blob/main/notebooks/data-preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify Project

I'm doing a data analysis project on my Spotify Extended Streaming History Data. The analyses I hope to do are:

1. Total listening time over time
2. Listening Time by Genre
3. Top artists / top tracks
4. Time-of-day listening patterns
5. Skip rate analysis

This Jupyter Notebook attempts to get the data cleaned and ready to use for visualisation in Tableau.

## Imports and Mounting

To begin, I will import everything I need and mount Google Drive so that I can access the files there:

In [ ]:
import os    # To enable the location of the project to be changed
import pandas as pd

from google.colab import drive    # To enable access to Google Drive files
from IPython.display import display    # To allow multiple outputs to be displayed in a single output cell

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting up the location to work in
I will now change the current working directory to be the project location:

In [ ]:
project_location = "/content/drive/MyDrive/spotify_project"
os.chdir(project_location)

## Gathering audio locations
My Spotify Extended Streaming History data is in the form of multiple JSON files in a folder. There are 13 files, file 0 to file 12. I will use this data along with data I will receive using Spotify's API. Before retrieving and integrating the data from Spotify's API, I will clean and prepare the Extended Streaming History data. I have already anonymised the data using I will now put all the locations of the JSON files into one list:

In [ ]:
audio_locations = [
    "data/working/Streaming_History_Audio_2017-2020_0.json",
    "data/working/Streaming_History_Audio_2020_1.json",
    "data/working/Streaming_History_Audio_2020_2.json",
    "data/working/Streaming_History_Audio_2020-2021_3.json",
    "data/working/Streaming_History_Audio_2021_4.json",
    "data/working/Streaming_History_Audio_2021_5.json",
    "data/working/Streaming_History_Audio_2021-2022_6.json",
    "data/working/Streaming_History_Audio_2022_7.json",
    "data/working/Streaming_History_Audio_2022-2023_8.json",
    "data/working/Streaming_History_Audio_2023_9.json",
    "data/working/Streaming_History_Audio_2023-2024_10.json",
    "data/working/Streaming_History_Audio_2024_11.json",
    "data/working/Streaming_History_Audio_2024-2025_12.json"
]

## Creating a dataframe for each audio location with relevant columns
While I do have the locations of the JSON files, the data from them has not been read. So, I will read each of them into its own dataframe object. Additionally, there is information I'm not concerned with, so I will keep only the information (columns) that I care about.

Here is an explanation of each of the relevant columns:

1. ts:
    - Date and time of when the stream ended in UTC format.<br><br>
2. ms_played:
    - For how many milliseconds the track was played.<br><br>
3. master_metadata_track_name:
    - Name of the track.<br><br>
4. master_metadata_album_artist_name:
    - Name of the artist, band, or podcast.<br><br>
5. master_metadata_album_album_name:
    - Name of the album of the track.<br><br>
6. spotify_track_uri:
    - A Spotify Track URI, that is identifying the unique music track.<br><br>
7. episode_name:
    - Name of the episode of the podcast.<br><br>
8. episode_show_name:
    - Name of the show of the podcast.<br><br>
9. spotify_episode_uri:
    - A Spotify Episode URI, that is identifying the unique podcast episode.<br><br>
10. audiobook_title:
    - Title of the audiobook.<br><br>
11. audiobook_uri:
    - A Spotify Audiobook URI, that is identifying the unique audiobook.<br><br>
12. audiobook_chapter_uri:
    - A Spotify Audiobook Chapter URI, that is identifying the unique audiobook chapter.<br><br>
13. audiobook_chapter_title:
    - Title of the audiobook chapter.<br><br>
14. reason_start:
    - Reason why the track started (e.g. previous track finished or you picked it from the playlist).<br><br>
15. reason_end:
    - Reason why the track ended (e.g. the track finished playing or you hit the next button).<br><br>
16. shuffle:
    - Whether shuffle mode was used when playing the track.<br><br>
17. skipped:
    - Information whether the user skipped to the next song.<br><br>

I am analysing tracks, not podcasts or audiobooks. But, I will inlcude columns for podcasts episodes and audiobooks in case there will be any podcast episodes or audiobooks in the dataframes. They will be removed. This will also account for the case of tracks that are also podcast episodes or audiobooks, if that's possible. If there are any, they will also be removed.

Finally, I will put all of the dataframes together into one list:

In [ ]:
audio_dfs = []
relevant_columns = [
    "ts",
    "ms_played",
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "spotify_track_uri",
    "episode_name",
    "episode_show_name",
    "spotify_episode_uri",
    "audiobook_title",
    "audiobook_uri",
    "audiobook_chapter_uri",
    "audiobook_chapter_title",
    "reason_start",
    "reason_end",
    "shuffle",
    "skipped"
]

# data_types = {
#     "master_metadata_track_name": "string",
#     "master_metadata_album_artist_name": "string",
#     "master_metadata_album_album_name": "string",
#     "spotify_track_uri": "string",
#     "reason_start": "string",
#     "reason_end": "string"
# }

for audio_location in audio_locations:    # For each JSON file, read it, select only the relevant columns, then add it to a list
    # audio_df = pd.read_json(audio_location, convert_dates=["ts"], dtype=data_types)    # Read each JSON file with "ts" as dates, and set specified dtypes
    audio_df = pd.read_json(audio_location)
    audio_df = audio_df[relevant_columns].copy()
    audio_dfs.append(audio_df)

## Learning general info about each dataframe
I need to have an idea of what each dataframe is like so that I can eventually put them together nicely and have them ready for analysis, but I'm unsure of the best way to do this. I need to start somewhere, so for each dataframe, I will look at some general information:

In [ ]:
for audio_df in audio_dfs:
    audio_df.info()
    print("\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15579 entries, 0 to 15578
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ts                                 15579 non-null  object 
 1   ms_played                          15579 non-null  int64  
 2   master_metadata_track_name         15579 non-null  object 
 3   master_metadata_album_artist_name  15579 non-null  object 
 4   master_metadata_album_album_name   15579 non-null  object 
 5   spotify_track_uri                  15579 non-null  object 
 6   episode_name                       0 non-null      float64
 7   episode_show_name                  0 non-null      float64
 8   spotify_episode_uri                0 non-null      float64
 9   audiobook_title                    0 non-null      float64
 10  audiobook_uri                      0 non-null      float64
 11  audiobook_chapter_uri              0 non-null      flo

## Checking that the column names of each dataframe are equal
Comparing the above general information for each dataframe against each other is not easy, so I'll also directly compare the columns of each dataframe:

In [ ]:
print("The column names are the same for dataframe")
for index, audio_df in enumerate(audio_dfs):
    if index != len(audio_dfs) - 1:    # Ensuring I don't go out of bounds for the list
        if audio_df.columns.equals(audio_dfs[index + 1].columns):
            print(f"{index} and {index + 1}? Yes.")
        else:
            print(f"{index} and {index + 1}? No.")

The column names are the same for dataframe
0 and 1? Yes.
1 and 2? Yes.
2 and 3? Yes.
3 and 4? Yes.
4 and 5? Yes.
5 and 6? Yes.
6 and 7? Yes.
7 and 8? Yes.
8 and 9? Yes.
9 and 10? Yes.
10 and 11? Yes.
11 and 12? Yes.


## Checking that the data types are consistent
Now, I will do that same as above, except for the data type of each column instead of for column names:

In [ ]:
print("The data types are consistent for dataframe")
for index, audio_df in enumerate(audio_dfs):
    if index != len(audio_dfs) - 1:    # Ensuring I don't go out of bounds for the list
        if audio_df.dtypes.equals(audio_dfs[index + 1].dtypes):
            print(f"{index} and {index + 1}? Yes.")
        else:
            print(f"{index} and {index + 1}? No.")

The data types are consistent for dataframe
0 and 1? Yes.
1 and 2? Yes.
2 and 3? Yes.
3 and 4? Yes.
4 and 5? Yes.
5 and 6? Yes.
6 and 7? Yes.
7 and 8? Yes.
8 and 9? Yes.
9 and 10? Yes.
10 and 11? Yes.
11 and 12? No.


## Investigating the difference in data types between dataframe 11 and 12

From above, I noticed that the data types are consistent everywhere, except for dataframe 11 and 12. Upon investigating the general information for dataframe 11 and 12 further above, I learned the differences in data type. It regards the episode_name, episode_show_name, and spotify_episode_uri columns. For dataframes 0 to 11, all values in each of these columns are null, and the data type is float64. But for dataframe 12, in each of these columns, there is 1 non-null value. Also, the data type of those columns is object instead of float64.

This may suggest there is a podcast episode in this dataframe, but I can't be sure. My analyses are for tracks, not podcast episodes, so if there are any episodes, they need to be removed.

## Checking distinct values for podcast episode columns and audiobook columns
So, there is podcast episode information in dataframe 12. I want to be absolutely certain there is no podcast or audiobook information in any of my dataframes, so I will go through each dataframe, and each of their podcast episode and audiobook columns to check for their unique values:

In [ ]:
questionable_columns = [
    "episode_name",
    "episode_show_name",
    "spotify_episode_uri",
    "audiobook_title",
    "audiobook_uri",
    "audiobook_chapter_uri",
    "audiobook_chapter_title"
]

for index, audio_df in enumerate(audio_dfs):
    for questionable_column in questionable_columns:
        print(f"DataFrame: \t{index}\nColumn: \t{questionable_column}")
        display(audio_df[questionable_column].value_counts())
        print("\n")

DataFrame: 	0
Column: 	episode_name


,count
episode_name,




DataFrame: 	0
Column: 	episode_show_name


,count
episode_show_name,




DataFrame: 	0
Column: 	spotify_episode_uri


...

,count
audiobook_uri,




DataFrame: 	12
Column: 	audiobook_chapter_uri


,count
audiobook_chapter_uri,




DataFrame: 	12
Column: 	audiobook_chapter_title


,count
audiobook_chapter_title,


## Learning about the episode information in dataframe 12

From the above, we see that for each dataframe, and for each of their podcast episode columns and audiobook columns, there are no values, except for dataframe 12. Here, there is one value in its episode_name column, one in episode_show_name, and one in spotify_episode_uri. This is decribed below.

Audio DataFrame: 12, Column: episode_name
episode_name
76. The ULEZ Blade Runners (the fuming suburbanites chopping down ULEZ cameras)    1
Name: count, dtype: int64

Audio DataFrame: 12, Column: episode_show_name
episode_show_name
The Screen Rot Podcast with Jacob and Jake    1
Name: count, dtype: int64

Audio DataFrame: 12, Column: spotify_episode_uri
spotify_episode_uri
spotify:episode:7AZTPXZTmoeGJKwXCgl6g6    1
Name: count, dtype: int64

## Finding the episode in dataframe 12

Now, I need to find where this information lives. I imagine that the above values all live in the same row, so I will look at the row in dataframe 12 that contains the value in the episode_name column to see if the episode_show_name value and the spotify_episode_uri are also in that row:

In [ ]:
audio_df_12 = audio_dfs[12]
episode_name_column = audio_df_12["episode_name"].copy()
audio_df_12[episode_name_column.notna()]

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped
5090,2025-03-09T20:37:49Z,0,None,None,None,None,76. The ULEZ Blade Runners (the fuming suburba...,The Screen Rot Podcast with Jacob and Jake,spotify:episode:7AZTPXZTmoeGJKwXCgl6g6,NaN,NaN,NaN,NaN,unknown,endplay,False,True


## Removing the episode

As we can see, the row that contains the episode_name value also contains the episode_show_name value and the spotify_episode_uri value. This row doesn't have track data, so it's not a track. I will safely remove this:

In [ ]:
audio_track_df_12 = audio_df_12[episode_name_column.isna()].copy()
audio_track_df_12

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped
0,2024-12-20T23:16:07Z,178051,Iron,Woodkid,Iron,spotify:track:0j1VVFif10Ib9s3U59adli,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False
1,2024-12-20T23:20:11Z,243304,Galaxy Wars,Evil Needle,Qualia,spotify:track:1FRnIWyCmEk38q1TsZhlCA,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False
2,2024-12-20T23:27:21Z,163377,Like this,박혜진 Park Hye Jin,How can I,spotify:track:7pY3HE16Ir4ZFTEOmosP57,None,None,None,NaN,NaN,NaN,NaN,trackdone,logout,True,False
3,2024-12-21T14:27:38Z,74626,Entreat,||||||||||||||||||||,Entreat,spotify:track:7BrBhLOpXQma7QpvnWxAvZ,None,None,None,NaN,NaN,NaN,NaN,trackdone,remote,False,False
4,2024-12-21T14:28:30Z,39973,Like this,박혜진 Park Hye Jin,How can I,spotify:track:7pY3HE16Ir4ZFTEOmosP57,None,None,None,NaN,NaN,NaN,NaN,remote,endplay,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12101,2025-08-16T19:31:58Z,36896,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,None,None,None,NaN,NaN,NaN,NaN,trackdone,backbtn,True,True
12102,2025-08-16T19:32:03Z,4563,Playstation,Ebi Soda,Ugh,spotify:track:5PETadLT856KzshN8ArpWl,None,None,None,NaN,NaN,NaN,NaN,backbtn,fwdbtn,True,True
12103,2025-08-16T19:38:07Z,364000,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,None,None,None,NaN,NaN,NaN,NaN,fwdbtn,trackdone,True,False
12104,2025-08-16T19:41:14Z,186227,Door Shutter,Brokenchord,Endless Transmission,spotify:track:2bBrirDuZf2mDbE6P7PCwh,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False


## Missing index

I noticed that in the above dataframe, there are 12105 row, but the first index is 0, and the last index is 12105. This means that the indices of the rows that come after the removed episode row aren't updated. We will do this:

In [ ]:
audio_track_df_12 = audio_track_df_12.reset_index(drop=True)    # drop=True, so the old index doesn't become a column
audio_track_df_12

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped
0,2024-12-20T23:16:07Z,178051,Iron,Woodkid,Iron,spotify:track:0j1VVFif10Ib9s3U59adli,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False
1,2024-12-20T23:20:11Z,243304,Galaxy Wars,Evil Needle,Qualia,spotify:track:1FRnIWyCmEk38q1TsZhlCA,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False
2,2024-12-20T23:27:21Z,163377,Like this,박혜진 Park Hye Jin,How can I,spotify:track:7pY3HE16Ir4ZFTEOmosP57,None,None,None,NaN,NaN,NaN,NaN,trackdone,logout,True,False
3,2024-12-21T14:27:38Z,74626,Entreat,||||||||||||||||||||,Entreat,spotify:track:7BrBhLOpXQma7QpvnWxAvZ,None,None,None,NaN,NaN,NaN,NaN,trackdone,remote,False,False
4,2024-12-21T14:28:30Z,39973,Like this,박혜진 Park Hye Jin,How can I,spotify:track:7pY3HE16Ir4ZFTEOmosP57,None,None,None,NaN,NaN,NaN,NaN,remote,endplay,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12100,2025-08-16T19:31:58Z,36896,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,None,None,None,NaN,NaN,NaN,NaN,trackdone,backbtn,True,True
12101,2025-08-16T19:32:03Z,4563,Playstation,Ebi Soda,Ugh,spotify:track:5PETadLT856KzshN8ArpWl,None,None,None,NaN,NaN,NaN,NaN,backbtn,fwdbtn,True,True
12102,2025-08-16T19:38:07Z,364000,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,None,None,None,NaN,NaN,NaN,NaN,fwdbtn,trackdone,True,False
12103,2025-08-16T19:41:14Z,186227,Door Shutter,Brokenchord,Endless Transmission,spotify:track:2bBrirDuZf2mDbE6P7PCwh,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False


## Assigning the neatened dataframe 12 back to the list of dataframes

Now, the final index is 12104, rather than 12105, meaning the indices have been updated to account for the gap. We now have to update the dataframe list so that its 12th entry is replaced with its version that's purely comprised of tracks.

In [ ]:
audio_dfs[12] = audio_track_df_12
audio_dfs[12]

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped
0,2024-12-20T23:16:07Z,178051,Iron,Woodkid,Iron,spotify:track:0j1VVFif10Ib9s3U59adli,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False
1,2024-12-20T23:20:11Z,243304,Galaxy Wars,Evil Needle,Qualia,spotify:track:1FRnIWyCmEk38q1TsZhlCA,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False
2,2024-12-20T23:27:21Z,163377,Like this,박혜진 Park Hye Jin,How can I,spotify:track:7pY3HE16Ir4ZFTEOmosP57,None,None,None,NaN,NaN,NaN,NaN,trackdone,logout,True,False
3,2024-12-21T14:27:38Z,74626,Entreat,||||||||||||||||||||,Entreat,spotify:track:7BrBhLOpXQma7QpvnWxAvZ,None,None,None,NaN,NaN,NaN,NaN,trackdone,remote,False,False
4,2024-12-21T14:28:30Z,39973,Like this,박혜진 Park Hye Jin,How can I,spotify:track:7pY3HE16Ir4ZFTEOmosP57,None,None,None,NaN,NaN,NaN,NaN,remote,endplay,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12100,2025-08-16T19:31:58Z,36896,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,None,None,None,NaN,NaN,NaN,NaN,trackdone,backbtn,True,True
12101,2025-08-16T19:32:03Z,4563,Playstation,Ebi Soda,Ugh,spotify:track:5PETadLT856KzshN8ArpWl,None,None,None,NaN,NaN,NaN,NaN,backbtn,fwdbtn,True,True
12102,2025-08-16T19:38:07Z,364000,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,None,None,None,NaN,NaN,NaN,NaN,fwdbtn,trackdone,True,False
12103,2025-08-16T19:41:14Z,186227,Door Shutter,Brokenchord,Endless Transmission,spotify:track:2bBrirDuZf2mDbE6P7PCwh,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,True,False


## Keeping the relevant columns of each dataframe

All the dataframes purely made up of tracks now, and all podcast episode columns and audiobook columns have no values. I feel safe changing my dataframes so they're only made of the columns that may be needed for my analyses:

In [ ]:
final_relevant_columns = [
    "ts",
    "ms_played",
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "spotify_track_uri",
    "reason_start",
    "reason_end",
    "shuffle",
    "skipped"
]

for index, audio_df in enumerate(audio_dfs):
    audio_dfs[index] = audio_df[final_relevant_columns].copy()

## Checking the general information of each dataframe again

Because of all of the changes I've made already, I want to be sure that each dataframe looks OK, so I'll look at their general information again.

In [ ]:
for audio_df in audio_dfs:
    audio_df.info()
    print("\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15579 entries, 0 to 15578
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ts                                 15579 non-null  object
 1   ms_played                          15579 non-null  int64 
 2   master_metadata_track_name         15579 non-null  object
 3   master_metadata_album_artist_name  15579 non-null  object
 4   master_metadata_album_album_name   15579 non-null  object
 5   spotify_track_uri                  15579 non-null  object
 6   reason_start                       15579 non-null  object
 7   reason_end                         15579 non-null  object
 8   shuffle                            15579 non-null  bool  
 9   skipped                            15579 non-null  bool  
dtypes: bool(2), int64(1), object(7)
memory usage: 1004.2+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15469 entries, 0 to 1

## Investigating the type of ts values

I noticed that object is the Dtype of all ts columns. I expected them to come out as dates, since ts, described by Spotify, is a date and time. So, I will learn about the type of all ts values to have a better idea of what is going on.

In [ ]:
for audio_df in audio_dfs:
    ts_values = audio_df["ts"].copy()
    ts_types = ts_values.apply(type)    # Use the type function to produce a Series of types for each ts value
    ts_types_count = ts_types.value_counts()    # Count how many of each type appears
    display(ts_types_count)

,count
ts,
<class 'str'>,15579


,count
ts,
<class 'str'>,15469


,count
ts,
<class 'str'>,15540


,count
ts,
<class 'str'>,15514


,count
ts,
<class 'str'>,15340


...

,count
ts,
<class 'str'>,16124


,count
ts,
<class 'str'>,16149


,count
ts,
<class 'str'>,16123


,count
ts,
<class 'str'>,16102


,count
ts,
<class 'str'>,12105


## Testing the ts strings to see if they're dates

We see that all of the ts values are strings. After looking at many displays of the data, the ts values seem to take the format of the following example: `2017-04-01T17:43:10Z`. This is an ISO 8601 datetime string. This can be parsed from a string to a datetime. While, I don't know what all ts values look like, I can attempt to parse all values into a datetime and be notified if any ts value is not a parseable timestamp using `pd.to_datetime(..., errors='raise')`:

In [ ]:
# audio_df

for audio_df in audio_dfs:    # Current point of work
    audio_df["ts"] = pd.to_datetime(audio_df["ts"], errors="raise", utc=True)    # utc=True ensures the datetime is timezone-aware and in UTC

## Investigating the type of ts values again

No error were raised. I will check the types of each ts value again like before.


In [ ]:
for audio_df in audio_dfs:
    ts_values = audio_df["ts"].copy()
    ts_types = ts_values.apply(type)    # Use the type function to produce a Series of types for each ts value
    ts_types_count = ts_types.value_counts()    # Count how many of each type appears
    display(ts_types_count)

,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,15579


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,15469


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,15540


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,15514


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,15340


...

,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,16124


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,16149


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,16123


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,16102


,count
ts,
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,12105


## Checking the data type of the ts columns themselves

So, we see that the type of values in the ts columns are all `pandas._libs.tslibs.timestamps.Timestamp`. This corresponds to `datetime64[ns, UTC]`, which is what we're hoping for. Now, I will check the columns that actually contain these values.

In [ ]:
for audio_df in audio_dfs:
  display(audio_df["ts"].dtype)

datetime64[ns, UTC]

datetime64[ns, UTC]

datetime64[ns, UTC]

datetime64[ns, UTC]

datetime64[ns, UTC]

...

datetime64[ns, UTC]

datetime64[ns, UTC]

datetime64[ns, UTC]

datetime64[ns, UTC]

datetime64[ns, UTC]

## Checking ts nulls

Now that I'm happy with the ts data type, I want to look at info about all the dataframes again.

In [ ]:
for audio_df in audio_dfs:
    audio_df.info()
    print("\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15579 entries, 0 to 15578
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 15579 non-null  datetime64[ns, UTC]
 1   ms_played                          15579 non-null  int64              
 2   master_metadata_track_name         15579 non-null  object             
 3   master_metadata_album_artist_name  15579 non-null  object             
 4   master_metadata_album_album_name   15579 non-null  object             
 5   spotify_track_uri                  15579 non-null  object             
 6   reason_start                       15579 non-null  object             
 7   reason_end                         15579 non-null  object             
 8   shuffle                            15579 non-null  bool               
 9   skipped                            15579 non-null 

## Object Dtype concerns

We can see that for the columns

- master_metadata_track_name,
- master_metadata_album_artist_name,
- master_metadata_album_album_name,
- spotify_track_uri,
- reason_start,
-  and reason_end,

their Dtype are all objects. I expected them to be strings. So, before converting the column Dtypes to string, I will first inspect if all the values in all these columns are strings.

In [ ]:
object_columns = [
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "spotify_track_uri",
    "reason_start",
    "reason_end"
]

for audio_df in audio_dfs:
  print(len(audio_df))
  for object_column in object_columns:
    display(audio_df[object_column].apply(type).value_counts())

15579


,count
master_metadata_track_name,
<class 'str'>,15579


,count
master_metadata_album_artist_name,
<class 'str'>,15579


,count
master_metadata_album_album_name,
<class 'str'>,15579


,count
spotify_track_uri,
<class 'str'>,15579


...

,count
master_metadata_album_artist_name,
<class 'str'>,12105


,count
master_metadata_album_album_name,
<class 'str'>,12105


,count
spotify_track_uri,
<class 'str'>,12105


,count
reason_start,
<class 'str'>,12105


,count
reason_end,
<class 'str'>,12105


## Converting the columns to string columns

From the above, we see that they are all strings. I will now convert those columns to string columns.

In [ ]:
for audio_df in audio_dfs:
  for object_column in object_columns:
    audio_df[object_column] = audio_df[object_column].astype("string")

## Checking the dataframes
I want to make sure that they colmumns are now string columns.

In [ ]:
for audio_df in audio_dfs:
  display(audio_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15579 entries, 0 to 15578
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 15579 non-null  datetime64[ns, UTC]
 1   ms_played                          15579 non-null  int64              
 2   master_metadata_track_name         15579 non-null  string             
 3   master_metadata_album_artist_name  15579 non-null  string             
 4   master_metadata_album_album_name   15579 non-null  string             
 5   spotify_track_uri                  15579 non-null  string             
 6   reason_start                       15579 non-null  string             
 7   reason_end                         15579 non-null  string             
 8   shuffle                            15579 non-null  bool               
 9   skipped                            15579 non-null 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15469 entries, 0 to 15468
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 15469 non-null  datetime64[ns, UTC]
 1   ms_played                          15469 non-null  int64              
 2   master_metadata_track_name         15469 non-null  string             
 3   master_metadata_album_artist_name  15469 non-null  string             
 4   master_metadata_album_album_name   15469 non-null  string             
 5   spotify_track_uri                  15469 non-null  string             
 6   reason_start                       15469 non-null  string             
 7   reason_end                         15469 non-null  string             
 8   shuffle                            15469 non-null  bool               
 9   skipped                            15469 non-null 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15540 entries, 0 to 15539
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 15540 non-null  datetime64[ns, UTC]
 1   ms_played                          15540 non-null  int64              
 2   master_metadata_track_name         15540 non-null  string             
 3   master_metadata_album_artist_name  15540 non-null  string             
 4   master_metadata_album_album_name   15540 non-null  string             
 5   spotify_track_uri                  15540 non-null  string             
 6   reason_start                       15540 non-null  string             
 7   reason_end                         15540 non-null  string             
 8   shuffle                            15540 non-null  bool               
 9   skipped                            15540 non-null 

...

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16102 entries, 0 to 16101
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 16102 non-null  datetime64[ns, UTC]
 1   ms_played                          16102 non-null  int64              
 2   master_metadata_track_name         16102 non-null  string             
 3   master_metadata_album_artist_name  16102 non-null  string             
 4   master_metadata_album_album_name   16102 non-null  string             
 5   spotify_track_uri                  16102 non-null  string             
 6   reason_start                       16102 non-null  string             
 7   reason_end                         16102 non-null  string             
 8   shuffle                            16102 non-null  bool               
 9   skipped                            16102 non-null 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12105 entries, 0 to 12104
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 12105 non-null  datetime64[ns, UTC]
 1   ms_played                          12105 non-null  int64              
 2   master_metadata_track_name         12105 non-null  string             
 3   master_metadata_album_artist_name  12105 non-null  string             
 4   master_metadata_album_album_name   12105 non-null  string             
 5   spotify_track_uri                  12105 non-null  string             
 6   reason_start                       12105 non-null  string             
 7   reason_end                         12105 non-null  string             
 8   shuffle                            12105 non-null  bool               
 9   skipped                            12105 non-null 

None

## Bringing the dataframes together into a single dataframe

I feel like I've done enough for the independent dataframes to allow me to bring them together without much issue. I will do this:

In [ ]:
actual_num_of_rows = 0

for audio_df in audio_dfs:
  actual_num_of_rows += len(audio_df)

print(f"The total number of rows in the combined dataframe should be {actual_num_of_rows}")

audio_df = pd.concat(audio_dfs, ignore_index=True)    # ignore_index automatically reindexes the concatenated dataframe
calulcated_num_of_rows = len(audio_df)
print(f"The calulcated number of rows is {calulcated_num_of_rows}")

audio_df

The total number of rows in the combined dataframe should be 200225
The calulcated number of rows is 200225


,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,shuffle,skipped
0,2017-04-01 17:43:10+00:00,36860,Romance for Cello and Piano,Frederick Delius,Romance for Cello and Piano,spotify:track:70nMUxP46mAcjW5iRpRiyN,clickrow,endplay,False,False
1,2017-04-01 19:52:27+00:00,36800,"Pavane, Op. 50",Gabriel Fauré,"Fauré: Requiem, Op. 48 / Pavane, Op. 50",spotify:track:2oqgnxjyNYItifLYl9MzaW,clickrow,endplay,False,False
2,2017-04-01 19:57:07+00:00,280933,Funkin' for Jamaica,Tom Browne,Funkin' For Jamaica,spotify:track:0xbkxTqSaW5blsYgRXpB5I,clickrow,trackdone,False,False
3,2017-04-01 20:01:58+00:00,290160,Love Rollercoaster,Ohio Players,Honey,spotify:track:1uhUJ5uTuHYdbitv3IsITR,trackdone,trackdone,False,False
4,2017-04-01 20:09:38+00:00,459533,The Payback,James Brown,The Payback,spotify:track:4u4Kb4W5mWtn9oz4A7SKK6,trackdone,endplay,False,False
...,...,...,...,...,...,...,...,...,...,...
200220,2025-08-16 19:31:58+00:00,36896,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,trackdone,backbtn,True,True
200221,2025-08-16 19:32:03+00:00,4563,Playstation,Ebi Soda,Ugh,spotify:track:5PETadLT856KzshN8ArpWl,backbtn,fwdbtn,True,True
200222,2025-08-16 19:38:07+00:00,364000,Fukurokouji,El Huervo,World's End,spotify:track:771NtAeF8EiFO4D4CQG2ij,fwdbtn,trackdone,True,False
200223,2025-08-16 19:41:14+00:00,186227,Door Shutter,Brokenchord,Endless Transmission,spotify:track:2bBrirDuZf2mDbE6P7PCwh,trackdone,trackdone,True,False


## Creation of derived columns

Now that my data feels clean and put together, I want to add more columns that will make analysis and visualisation easier. This includes the following:

### Time-Based Derived Columns

- date
- year
- month
- year_month
- hour
- day_of_week

### Playback & Behaviour Columns

- listening_hours
- track_play_count
- skip_flag
- completed_flag
- valid_play

### Genre Enrichment Columns

- genre
- genre_group

I will start with the Time-Based Derived Columns

In [ ]:
# date
audio_df["date"] = audio_df["ts"].dt.date
print(audio_df.loc[0, "date"])
print(f"The type of a \"date\" value is {type(audio_df.loc[0, "date"])}")

# year
audio_df["year"] = audio_df["ts"].dt.year
print(audio_df.loc[0, "year"])
print(f"The type of a \"year\" value is {type(audio_df.loc[0, "year"])}")

# month
audio_df["month"] = audio_df["ts"].dt.month
print(audio_df.loc[0, "month"])
print(f"The type of a \"month\" value is {type(audio_df.loc[0, "month"])}")

# year_month
# month_padded = audio_df["month"].astype(str)
if audio_df["month"] < 10:
  month_string = "0" + audio_df["month"].astype(str)
else:
  month_string = audio_df["month"].astype(str)

audio_df["year_month"] = audio_df["year"].astype(str) + "-" + month_string

# audio_df["year_month"] = audio_df["year"].astype(str) + "-" + audio_df["month"].astype(str)
print(audio_df.loc[0, "year_month"])
print(f"The type of a \"year_month\" value is {type(audio_df.loc[0, "year_month"])}")

# hour

# day_of_week


2017-04-01
The type of a "date" value is <class 'datetime.date'>
2017
The type of a "year" value is <class 'numpy.int32'>
4
The type of a "month" value is <class 'numpy.int32'>


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Personal Note:

I'm following the Python book, I'm on Chapter 7: Data Cleaning and Preparation. The subchapter is 7.2: Data Transformation. Within that, I'm looking at Transforming Data Using a Function or Mapping. I'm interested in possibly adding derived columns for date, month, year, hour, and day_of_week. That's why this part of the book I'm on is interesting. However, I'm wondering if I should change the data types of my time-based columns before doing this, and how. So, I'm going through the book and other resources to find out more about data types.

## Checking data types in each column

I'm not satisfied with that I've got "object" as the Dtype of some of my columns, when they should be a string type. So, I want to check the contents of these columns, to make sure that all of the data inside are all strings, before making the Dtype of their columns into a string dtype:

In [ ]:
# object_columns = [
#     "master_metadata_track_name",
#     "master_metadata_album_artist_name",
#     "master_metadata_album_album_name",
#     "spotify_track_uri",
#     "reason_start",
#     "reason_end"
# ]

# for index, audio_df in enumerate(audio_dfs):
#     for column in object_columns:
#         if audio_df[column].apply(lambda x: isinstance(x, str)).all():    # if all values in the current column are strings
#             print(f"For dateframe {index}, column {column},")
#             print(f"all values are strings")
#             print("\n")
#         else:
#             print(f"For dateframe {index}, column {column},")
#             print(f"not all values are strings")
#             print("\n")

## Checking distinct `reason_end` values

In [ ]:
# audio_0_df["reason_end"].value_counts()